<a href="https://colab.research.google.com/github/Cheimooh/DataMiningProject/blob/main/DM_Projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sklearn
import numpy as np

df = pd.read_csv("./data/googleplaystore.csv")
df.info()

### Colonnes

a première vue, il n'y a qu'une colonne qui contient des données numériques. L'idée est donc de vérifier quelles colonnes il serait possible de tranformer en données numériques afin d'avoir plus de matières pour les analyses suivantes


#### Size (ok)

In [ ]:
df["Size"]

Dans le cas présent, Size est un objet, qui pourrait être convertis en une valeur numérique

#### Category (ok)

In [ ]:
df["Category"]

Category, pourrait être convertie en données numériques, par ex 1 pour ART_AND_DESIGN, 2 pour FAMILLY etc

#### Reviews (ok)

In [ ]:
df["Reviews"]

Reviews correspond au nombre de reviews, il faut donc le convertir en valeur numérique

#### Installs (ok)

In [ ]:
df['Installs'].unique()

Correspond au nombre d'installations (ordre de grandeur)
Il suffit d'enlever le +

#### Type (abandonnée)

In [ ]:
df['Type'].unique()

Semble correspondre au type de l'aplication (payante ou non). Peut etre convertie en valeur booleenne permettant de determiner si une app est payante ou non



#### Price (ok)

In [ ]:
df["Price"].unique()

Valeur numérique, il faut enlever le $$$$

#### Content Rating (ok)

In [ ]:
df['Content Rating'].unique()

on peut transformer tout ca en une valeur numérique



#### Genres (ok)

In [ ]:
df["Genres"].unique()

a voir

#### Last Updated (ok)

In [ ]:
df["Last Updated"]

#### Current Ver (abandonnée)

In [ ]:
df["Current Ver"]

#### Android Ver (abandon)

ne sert a rien car trop de lignes a enlever, on peut se baser sur la colonne "last updated" pour savoir si une app est maintenue ou pas


In [ ]:
from pprint import pprint
pprint(df["Android Ver"].unique())
print(df["Android Ver"].isna().sum())
print(df["Android Ver"].isnull().sum())

### Conversion des colonnes en numérique

In [ ]:
def col_to_num(col):
  df[col] = pd.to_numeric(df[col], errors="coerce")
  print(df[col])

#### Colonne "Size" (à faire)

In [ ]:
print(df["Size"].unique())

In [ ]:
# Supprime les lignes où les valeurs sont pas interprétables (1,000+ et Varies with device)
df = df.drop(df[df["Size"].str.contains('\+', regex=True)].index)
df = df.drop(df[df["Size"].str.contains('Varies', regex=True)].index)
print(df["Size"].unique())
df.info()

In [ ]:
df["Size"] = df["Size"].str.replace('M','000000')
df["Size"] = df["Size"].str.replace('k', '000')
print(df["Size"].unique())

In [ ]:
df['Size'] = df['Size'].apply(lambda x: x[:-1] if '.' in x else x)
df["Size"] = df["Size"].str.replace('.', '')
print(df["Size"].unique())

In [ ]:
col_to_num("Size")

#### Colonne "Installs"

In [ ]:
df["Installs"] = df["Installs"].str.replace('+', '')
df["Installs"] = df["Installs"].str.replace(',', '')

In [ ]:
col_to_num("Installs")

#### Colonne "Price"


In [ ]:
df["Price"] = df["Price"].str.replace('$', '')
print(df["Price"].unique())

In [ ]:
col_to_num("Price")

#### Colonne "Reviews"

In [ ]:
print(df["Reviews"].unique())

In [ ]:
col_to_num("Reviews")

### Enum

In [ ]:
def dict_from_cat(col):
  return {valeur_unique: indice for indice, valeur_unique in enumerate(df[col].unique())}

In [ ]:
def dict_to_float(col, dictionary):
  df[col] = df[col].map(dictionary)
  col_to_num(col)

#### Colonne "Category"

In [ ]:
df["Category"].unique()

In [ ]:
len(df["Category"].unique())

In [ ]:
dict_category = dict_from_cat("Category")
dict_to_float("Category", dict_category)

#### Colonne "Content Rating"

In [ ]:
df["Content Rating"].unique()

In [ ]:
dict_contentRating = dict_from_cat("Content Rating")
dict_to_float("Content Rating", dict_contentRating)

In [ ]:
df["Content Rating"].unique()

#### Colonne "Genres"
Probleme, cette colonne contient plusieurs infos, on va donc la split en 2 colonnes avant de la passée en valeurs numériques

In [ ]:
df[["Genres", "GenreSecondaire"]] = df["Genres"].str.split(";", expand=True)
df = df.rename(columns={"Genres" : "GenrePrincipal"})
df["GenreSecondaire"]

##### Genre Principal

In [ ]:
dict_genrePrincipal = dict_from_cat("GenrePrincipal")
dict_to_float("GenrePrincipal", dict_genrePrincipal)

##### Genre secondaire

In [ ]:
dict_genreSecondaire = dict_from_cat("GenreSecondaire")
dict_to_float("GenreSecondaire", dict_genreSecondaire)

### Conversion colonne date

In [ ]:
#df['Last Updated'] = df['Last Updated'].str.replace(' [0-9]+,', '', regex=True)
df["Last Updated"].unique()
df["Last Updated"] = df["Last Updated"].replace('1.0.19', 'NaN')
df['Last Updated'] = pd.to_datetime(df['Last Updated'], format='%B %d, %Y')
df["Last Updated"].dt.date

### Colonnes abandonnées

In [ ]:
#df = df.drop(columns=["color", "type", "condition"])
df = df.drop(columns=["Type", "Current Ver", "Android Ver"])
df.info()

In [ ]:
df.info()

In [ ]:
df.to_csv("./data/clean_dataset_googleplaystore.csv", index=False)

#### Colonne "Type"

La colonne "Prix" nous donne déjà l'infomation si le produit est gratuit ou non.

#### Colonne "Current Ver"

Un peu inutile, dépend des applications.